### Isentropic Flow Calculator

In [1]:
import math
import ipywidgets as widgets
from IPython.display import display

gamma = 1.4
p_star_ratio = 0.528
rho_star_ratio = 0.634
T_star_ratio = 0.833

def mach_from_pressure_ratio(p_pt):
    return math.sqrt((2 / (gamma - 1)) * ((p_pt**(-(gamma - 1) / gamma)) - 1))

def mach_from_temp_ratio(T_Tt):
    return math.sqrt((2 / (gamma - 1)) * (1 / T_Tt - 1))

def mach_from_density_ratio(rho_rhot):
    return math.sqrt((2 / (gamma - 1)) * (rho_rhot**(-(gamma - 1)) - 1))

def mach_from_area_ratio(A_A_star):
    def f(M):
        return (1 / M) * ((2 / (gamma + 1)) * (1 + (gamma - 1) / 2 * M**2))**((gamma + 1) / (2 * (gamma - 1))) - A_A_star

    from scipy.optimize import fsolve
    M_subsonic = fsolve(f, 0.5)[0]
    M_supersonic = fsolve(f, 2.0)[0]
    return M_subsonic, M_supersonic

def mach_angle(M):
    if M < 1:
        return None
    return math.degrees(math.asin(1 / M))

def prandtl_meyer_angle(M):
    if M <= 1:
        return None
    term1 = math.sqrt((gamma + 1) / (gamma - 1))
    term2 = math.atan(math.sqrt(((gamma - 1) / (gamma + 1)) * (M**2 - 1)))
    term3 = math.atan(math.sqrt(M**2 - 1))
    return math.degrees(term1 * term2 - term3)

def calculate_properties(M):
    p_pt = (1 + (gamma - 1) / 2 * M**2)**(-gamma / (gamma - 1))
    T_Tt = (1 + (gamma - 1) / 2 * M**2)**-1
    rho_rhot = (1 + (gamma - 1) / 2 * M**2)**(-1 / (gamma - 1))
    A_A_star = (1 / M) * ((2 / (gamma + 1)) * (1 + (gamma - 1) / 2 * M**2))**((gamma + 1) / (2 * (gamma - 1)))

    p_p_star = p_pt / p_star_ratio
    rho_rho_star = rho_rhot / rho_star_ratio
    T_T_star = T_Tt / T_star_ratio
    mach_angle_val = mach_angle(M)
    prandtl_meyer_val = prandtl_meyer_angle(M)

    return {
        'Mach number (M)': M,
        'Pressure ratio (p/pt)': p_pt,
        'Temperature ratio (T/Tt)': T_Tt,
        'Density ratio (rho/rhot)': rho_rhot,
        'Area ratio (A/A*)': A_A_star,
        'Pressure ratio (p/p*)': p_p_star,
        'Density ratio (rho/rho*)': rho_rho_star,
        'Temperature ratio (T/T*)': T_T_star,
        'Mach angle (μ)': mach_angle_val,
        'Prandtl-Meyer angle (ν)': prandtl_meyer_val
    }

def on_input_change(change):
    input_type = dropdown.value
    input_value = input_box.value

    if input_value.strip() == '':
        results.value = 'Results will appear here...'
        return

    try:
        input_value = float(input_value)

        if input_type == 'Area ratio (A/A*)':
            M_sub, M_sup = mach_from_area_ratio(input_value)

            subsonic_properties = calculate_properties(M_sub)
            supersonic_properties = calculate_properties(M_sup)

            def format_properties(props):
                return '\n'.join([f"{key}: {value:.4f}" if value is not None else f"{key}: N/A" for key, value in props.items()])

            results.value = f"""
            Subsonic Results:
            {format_properties(subsonic_properties)}

            Supersonic Results:
            {format_properties(supersonic_properties)}
            """
        else:
            if input_type == 'Mach number (M)':
                M = input_value
            elif input_type == 'Pressure ratio (p/pt)':
                M = mach_from_pressure_ratio(input_value)
            elif input_type == 'Temperature ratio (T/Tt)':
                M = mach_from_temp_ratio(input_value)
            elif input_type == 'Density ratio (rho/rhot)':
                M = mach_from_density_ratio(input_value)
            elif input_type == 'Mach angle (μ)':
                M = 1 / math.sin(math.radians(input_value))
            elif input_type == 'Prandtl-Meyer angle (ν)':
                from scipy.optimize import fsolve
                def prandtl_meyer_eq(M):
                    return prandtl_meyer_angle(M) - input_value
                M = fsolve(prandtl_meyer_eq, 2.0)[0]

            properties = calculate_properties(M)

            def format_properties(props):
                return '\n'.join([f"{key}: {value:.4f}" if value is not None else f"{key}: N/A" for key, value in props.items()])

            results.value = format_properties(properties)
    except:
        results.value = 'Invalid input'

dropdown = widgets.Dropdown(
    options=['Mach number (M)', 'Pressure ratio (p/pt)', 'Temperature ratio (T/Tt)', 'Density ratio (rho/rhot)', 'Area ratio (A/A*)', 'Mach angle (μ)', 'Prandtl-Meyer angle (ν)'],
    description='Input type:'
)

input_box = widgets.Text(
    value='1.0',
    description='Input value:',
    disabled=False
)

results = widgets.Textarea(
    value='Results will appear here...',
    description='Results:',
    layout={'width': '600px', 'height': '500px'}
)

dropdown.observe(on_input_change, names='value')
input_box.observe(on_input_change, names='value')

display(dropdown, input_box, results)

Dropdown(description='Input type:', options=('Mach number (M)', 'Pressure ratio (p/pt)', 'Temperature ratio (T…

Text(value='1.0', description='Input value:')

Textarea(value='Results will appear here...', description='Results:', layout=Layout(height='500px', width='600…

### Normal Shock Flow Calculator

In [2]:
import numpy as np
import ipywidgets as widgets
from IPython.display import display

gamma = 1.4

def calculate_M2(M1):
    """ Calculate downstream Mach number M2 from upstream Mach number M1. """
    if M1 <= 1:
        raise ValueError("Upstream Mach number M1 must be greater than 1 for a normal shock.")
    return np.sqrt((M1**2 * (gamma - 1) + 2) / (2 * gamma * M1**2 - (gamma - 1)))

def calculate_pressure_ratio(M1):
    """ Calculate p2/p1 from M1. """
    return (2 * gamma * M1**2 - (gamma - 1)) / (gamma + 1)

def calculate_density_ratio(M1):
    """ Calculate rho2/rho1 from M1. """
    return ((gamma + 1) * M1**2) / ((gamma - 1) * M1**2 + 2)

def calculate_temperature_ratio(M1):
    """ Calculate T2/T1 from M1. """
    return calculate_pressure_ratio(M1) / calculate_density_ratio(M1)

def calculate_stagnation_pressure_ratio(M1):
    """ Calculate p02/p01 from M1. """
    term1 = ((gamma + 1) * M1**2 / (2 + (gamma - 1) * M1**2))**(gamma / (gamma - 1))
    term2 = (1 / ((2 * gamma * M1**2 / (gamma + 1)) - ((gamma - 1) / (gamma + 1))))**(1 / (gamma - 1))
    return term1 * term2

def calculate_all_from_M1(M1):
    M2 = calculate_M2(M1)
    p2_p1 = calculate_pressure_ratio(M1)
    rho2_rho1 = calculate_density_ratio(M1)
    T2_T1 = calculate_temperature_ratio(M1)
    p02_p01 = calculate_stagnation_pressure_ratio(M1)
    return M2, p2_p1, rho2_rho1, T2_T1, p02_p01

def calculate_M1_from_pressure_ratio(p2_p1):
    """ Calculate M1 from p2/p1. """
    M1 = np.sqrt((p2_p1 * (gamma + 1) + (gamma - 1)) / (2 * gamma))
    return M1

def calculate_M1_from_density_ratio(rho2_rho1):
    """ Calculate M1 from rho2/rho1. """
    M1 = np.sqrt((rho2_rho1 * (gamma - 1) + 2) / (2 * (gamma + 1)))
    return M1

def calculate_M1_from_temperature_ratio(t):
    """ Calculate M1 from T2/T1. """
    G = ((gamma + 1)/(gamma - 1))
    b = G*(1-t)
    p2_p1 = (-b + np.sqrt(b**2 + 4*t))/2
    return calculate_M1_from_pressure_ratio(p2_p1)

def calculate_M1_from_p1_p02(p1_p02):
    """ Calculate M1 from p1/p02. """
    return np.sqrt(2 / (gamma - 1) * (1 / (p1_p02 ** ((gamma - 1) / gamma)) - 1))

def calculate_M1_from_M2(M2):
    """ Calculate upstream Mach number M1 from downstream Mach number M2. """
    if M2 >= 1:
        raise ValueError("Downstream Mach number M2 must be less than 1 for a normal shock.")
    return np.sqrt((M2**2 * (gamma - 1) + 2) / (2 * gamma * M2**2 - (gamma - 1)))

def calculate_normal_shock_properties(input_type, input_value):
    input_value = float(input_value)

    if input_type == 'Upstream Mach number (M1)':
        M1 = input_value
        if M1 <= 1:
            raise ValueError("Upstream Mach number M1 must be greater than 1 for a normal shock.")
    elif input_type == 'Pressure ratio (p2/p1)':
        M1 = calculate_M1_from_pressure_ratio(input_value)
        if M1 <= 1:
            raise ValueError("Calculated M1 must be greater than 1 for a normal shock.")
    elif input_type == 'Density ratio (rho2/rho1)':
        M1 = calculate_M1_from_density_ratio(input_value)
        if M1 <= 1:
            raise ValueError("Calculated M1 must be greater than 1 for a normal shock.")
    elif input_type == 'Temperature ratio (T2/T1)':
        M1 = calculate_M1_from_temperature_ratio(input_value)
        if M1 <= 1:
            raise ValueError("Calculated M1 must be greater than 1 for a normal shock.")
    elif input_type == 'Downstream Mach number (M2)':
        M2 = input_value
        if M2 >= 1:
            raise ValueError("Downstream Mach number M2 must be less than 1 for a normal shock.")
        M1 = calculate_M1_from_M2(M2)
    elif input_type == 'p1/p02 (Pressure ratio)':
        M1 = calculate_M1_from_p1_p02(input_value)
        if M1 <= 1:
            raise ValueError("Calculated M1 must be greater than 1 for a normal shock.")
    else:
        raise ValueError("Invalid input type selected.")

    M2, p2_p1, rho2_rho1, T2_T1, p02_p01 = calculate_all_from_M1(M1)

    p1_p02 = (1 / p02_p01)/((1 + ((gamma - 1)/2)*(M1**2)))**((gamma)/(gamma-1))

    return M1, M2, p2_p1, rho2_rho1, T2_T1, p02_p01, p1_p02

input_type_dropdown = widgets.Dropdown(
    options=['Upstream Mach number (M1)', 'Pressure ratio (p2/p1)', 'Density ratio (rho2/rho1)',
             'Temperature ratio (T2/T1)', 'Downstream Mach number (M2)', 'p1/p02 (Pressure ratio)'],
    value='Upstream Mach number (M1)',
    description='Input Type:'
)

input_value_text = widgets.Text(
    value='2.0',
    description='Input Value:'
)

results = widgets.Textarea(
    value='Results will appear here...',
    description='Results:',
    layout={'width': '500px', 'height': '300px'}
)

def update_output(change=None):
    input_type = input_type_dropdown.value
    input_value = input_value_text.value.strip()

    if input_value == '':
        results.value = 'Results will appear here...'
        return

    try:
        M1, M2, p2_p1, rho2_rho1, T2_T1, p02_p01, p1_p02 = calculate_normal_shock_properties(input_type, input_value)

        results.value = f"""
        M1 (Upstream Mach number): {M1:.4f}
        M2 (Downstream Mach number): {M2:.4f}
        p2/p1 (Pressure ratio): {p2_p1:.4f}
        rho2/rho1 (Density ratio): {rho2_rho1:.4f}
        T2/T1 (Temperature ratio): {T2_T1:.4f}
        p02/p01 (Stagnation pressure ratio): {p02_p01:.4f}
        p1/p02 (Pressure ratio): {p1_p02:.4f}
        """
    except ValueError as e:
        results.value = str(e)
    except Exception as e:
        results.value = 'Invalid input: ' + str(e)

input_type_dropdown.observe(update_output, names='value')
input_value_text.observe(update_output, names='value')

display(input_type_dropdown, input_value_text, results)

Dropdown(description='Input Type:', options=('Upstream Mach number (M1)', 'Pressure ratio (p2/p1)', 'Density r…

Text(value='2.0', description='Input Value:')

Textarea(value='Results will appear here...', description='Results:', layout=Layout(height='300px', width='500…

### Oblique Shock Flow Calculator

In [3]:
import numpy as np
import math
import ipywidgets as widgets
from IPython.display import display

def deg_to_rad(deg):
    return deg * np.pi / 180

def rad_to_deg(rad):
    return rad * 180 / np.pi

def oblique_shock_eq(beta, delta, M1, gamma=1.4):
    beta = deg_to_rad(beta)
    delta = deg_to_rad(delta)
    left_side = math.tan(delta)
    right_side = (2 * (1 / math.tan(beta)) * (M1**2 * math.sin(beta)**2 - 1)) / (2 + M1**2 * (gamma + math.cos(2 * beta)))
    return left_side - right_side

def bisection_method(func, delta, M1, a, b, tol=1e-6, max_iter=100):
    if func(a, delta, M1) * func(b, delta, M1) >= 0:
        print("Bisection method fails. No root bracketed.")
        return None

    iteration = 0
    while (b - a) / 2 > tol and iteration < max_iter:
        mid = (a + b) / 2
        if func(mid, delta, M1) == 0:
            return mid
        elif func(a, delta, M1) * func(mid, delta, M1) < 0:
            b = mid
        else:
            a = mid
        iteration += 1

    return (a + b) / 2

def find_beta_bisection(delta, M1, gamma=1.4):
    mach_angle = np.degrees(np.arcsin(1 / M1))
    weak_lower_bound = mach_angle
    weak_upper_bound = 45.0

    strong_lower_bound = 45.0
    strong_upper_bound = 90.0

    beta_weak = bisection_method(oblique_shock_eq, delta, M1, weak_lower_bound, weak_upper_bound)
    beta_strong = bisection_method(oblique_shock_eq, delta, M1, strong_lower_bound, strong_upper_bound)

    return beta_weak, beta_strong

gamma = 1.4

def calculate_Mn1(M, s):
    return M * np.sin(np.radians(s))

def calculate_Mn2(Mn1):
    if Mn1 <= 1:
        raise ValueError("Upstream Mach number M1 must be greater than 1 for a normal shock.")
    return np.sqrt((Mn1**2 * (gamma - 1) + 2) / (2 * gamma * Mn1**2 - (gamma - 1)))

def calculate_temperature_ratio(M, s):
    Mn1 = calculate_Mn1(M, s)
    return ((2 * gamma * Mn1**2 - (gamma - 1)) * ((gamma - 1) * Mn1**2 + 2)) / (((gamma + 1) * Mn1)**2)

def calculate_pressure_ratio(M, s):
    Mn1 = calculate_Mn1(M, s)
    return (2 * gamma * Mn1**2 - (gamma - 1)) / (gamma + 1)

def calculate_density_ratio(M, s):
    Mn1 = calculate_Mn1(M, s)
    return ((gamma + 1) * Mn1**2) / ((gamma - 1) * Mn1**2 + 2)

def calculate_total_pressure_ratio(M, s):
    Mn1 = calculate_Mn1(M, s)
    term1 = ((gamma + 1) * Mn1**2 / ((gamma - 1) * Mn1**2 + 2))**(gamma / (gamma - 1))
    term2 = ((gamma + 1) / (2 * gamma * Mn1**2 - (gamma - 1)))**(1 / (gamma - 1))
    return term1 * term2

def calculate_turn_angle(M, s):
    return np.degrees(np.arctan((2 * (1 / math.tan(np.radians(s))) * (M**2 * np.sin(np.radians(s))**2 - 1)) / (2 + M**2 * (gamma + np.cos(2 * np.radians(s))))))

def calculate_oblique_shock_properties(M, input_type, input_value):
    input_value = float(input_value)

    a, s, Mn1 = None, None, None

    if input_type == 'Turn angle (weak shock)':
        a = input_value
        beta_weak, _ = find_beta_bisection(a, M)
        s = beta_weak
        Mn1 = calculate_Mn1(M, s)
    elif input_type == 'Wave angle (s)':
        s = input_value
        Mn1 = calculate_Mn1(M, s)
        a = calculate_turn_angle(M, s)
    elif input_type == 'Normal Mach number (Mn1 = M*sin(s))':
        Mn1 = input_value
        s = np.degrees(np.arcsin(Mn1 / M))
        a = calculate_turn_angle(M, s)
    elif input_type == 'Turn angle (strong shock)':
        a = input_value
        _, beta_strong = find_beta_bisection(a, M)
        s = beta_strong
        Mn1 = calculate_Mn1(M, s)

    T2_T1 = calculate_temperature_ratio(M, s) if s is not None else None
    p2_p1 = calculate_pressure_ratio(M, s) if s is not None else None
    rho2_rho1 = calculate_density_ratio(M, s) if s is not None else None
    Pt2_Pt1 = calculate_total_pressure_ratio(M, s) if s is not None else None
    Mn2 = calculate_Mn2(Mn1) if Mn1 is not None else None
    M2 = Mn2 / np.sin(np.radians(s - a)) if Mn2 is not None and s is not None and a is not None else None

    return rad_to_deg(a), s, Mn1, T2_T1, p2_p1, rho2_rho1, Pt2_Pt1, Mn2, M2

input_type_dropdown = widgets.Dropdown(
    options=['Turn angle (weak shock)', 'Turn angle (strong shock)', 'Wave angle (s)', 'Normal Mach number (Mn1 = M*sin(s))'],
    value='Turn angle (weak shock)',
    description='Input Type:',
)

M_value_text = widgets.Text(
    value='2.0',
    description='M1 (Upstream Mach Number):',
)

input_value_text = widgets.Text(
    value='0.5',
    description='Input Value:',
)

results = widgets.Textarea(
    value='Results will appear here...',
    description='Results:',
    layout={'width': '500px', 'height': '300px'}
)

def update_output(M_value, input_type, input_value):
    if M_value.strip() == '' or input_value.strip() == '':
        results.value = "Please enter valid numeric values."
        return

    try:
        M = float(M_value)
        input_value = float(input_value)

        a, s, Mn1, T2_T1, p2_p1, rho2_rho1, Pt2_Pt1, Mn2, M2 = calculate_oblique_shock_properties(M, input_type, input_value)

        output = f"Turn angle (a) in degrees: {a*2*np.pi/360 if a is not None else 'N/A'}\n"
        output += f"Wave angle (s): {s if s is not None else 'N/A'}\n"
        output += f"Normal Mach number (Mn1): {Mn1 if Mn1 is not None else 'N/A'}\n"
        output += f"Temperature Ratio (T2/T1): {T2_T1 if T2_T1 is not None else 'N/A'}\n"
        output += f"Pressure Ratio (p2/p1): {p2_p1 if p2_p1 is not None else 'N/A'}\n"
        output += f"Density Ratio (rho2/rho1): {rho2_rho1 if rho2_rho1 is not None else 'N/A'}\n"
        output += f"Total Pressure Ratio (Pt2/Pt1): {Pt2_Pt1 if Pt2_Pt1 is not None else 'N/A'}\n"
        output += f"Normal Mach number after shock (Mn2): {Mn2 if Mn2 is not None else 'N/A'}\n"
        output += f"Mach number after shock (M2): {M2 if M2 is not None else 'N/A'}\n"

        results.value = output
    except ValueError:
        results.value = "Please enter valid numeric values."

widgets.interactive(update_output, M_value=M_value_text, input_type=input_type_dropdown, input_value=input_value_text, results=results)

display(M_value_text, input_type_dropdown, input_value_text, results)

Text(value='2.0', description='M1 (Upstream Mach Number):')

Dropdown(description='Input Type:', options=('Turn angle (weak shock)', 'Turn angle (strong shock)', 'Wave ang…

Text(value='0.5', description='Input Value:')

Textarea(value='Results will appear here...', description='Results:', layout=Layout(height='300px', width='500…